In [25]:
!pip install mlxtend
!pip install apyori graphviz
!pip install pyfpgrowth
import pyfpgrowth
!grep -r "def mine_patterns"

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Assignment 3.ipynb:      "Assignment 3.ipynb:    \"!grep -r \\\"def mine_patterns\\\"\"\n"
Assignment 3.ipynb:    "!grep -r \"def mine_patterns\""
Assignment 3.ipynb:    "def mine_patterns(node, prefix, header_table, min_support, patterns):\n",
Assignment 3.ipynb:    "def mine_patterns(node, prefix, header_table, min_support, patterns):\n",


In [26]:
#Create a sample dataset
!pip install pandas
import pandas as pd

#Create a sample dataset:
data = pd.read_csv('california_housing_train.csv')
data = pd.DataFrame({
    "longitude": [1, 2, 3, 4, 5],
    "latitude": [10, 20, 30, 40, 50],
    # ... other columns
})

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [27]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
import graphviz

In [28]:
#Convert the dataset to a transaction format:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
df_encoded

,a,d,e,g,i,l,n,o,t,u
0,False,True,True,True,True,True,True,True,True,True
1,True,True,True,False,True,True,False,False,True,True
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [29]:
#Apply the Apriori algorithm:
frequent_itemsets_apriori = apriori(df_encoded, min_support=0.33, use_colnames=True)
frequent_itemsets_apriori

,support,itemsets
0,0.4,(d)
1,0.4,(e)
2,0.4,(i)
3,0.4,(l)
4,0.4,(t)
...,...,...
58,0.4,"(d, t, e, i, u)"
59,0.4,"(d, t, e, l, u)"
60,0.4,"(d, t, i, l, u)"
61,0.4,"(t, e, i, l, u)"


In [30]:
#Apply the FP-growth algorithm:
frequent_itemsets_fpgrowth = fpgrowth(df_encoded, min_support=0.33, use_colnames=True)
frequent_itemsets_fpgrowth

,support,itemsets
0,0.4,(u)
1,0.4,(t)
2,0.4,(l)
3,0.4,(i)
4,0.4,(e)
...,...,...
58,0.4,"(d, t, e, l, u)"
59,0.4,"(d, t, e, i, u)"
60,0.4,"(d, e, i, l, u)"
61,0.4,"(d, t, e, i, l)"


In [44]:
#Construction of FP-Tree class Node:
def __init__(self, item, count, parent):
    self.item = item 
    self.count = count 
    self.parent = parent
    self.children = {}
def build_tree(data, min_support):
    header_table = {} 
    for index, row in data.iterrows(): 
      for item in row: 
        header_table[item] = header_table.get(item, 0) + 1
        for k in list(header_table):
            if header_table[k] < min_support:
                del header_table[k]
                frequent_items = list(header_table.keys()) 
                frequent_items.sort(key=lambda x: header_table[x], reverse=True) 
                root = Node("Null", 1, None)
                for index, row in data.iterrows():
                    ordered_items = [item for item in frequent_items if item in row]
                    if ordered_items: 
                        insert_tree(ordered_items, root, header_table, 1)
                        # Ensure 'Null' is in header_table 
                        if 'Null' not in header_table: 
                            header_table['Null'] = (0, None)
                            return root, header_table
                            def insert_tree(items, node, header_table, count):
                                if not items:
                                    return
                                    if items[0] in node.children: 
                                        node.children[items[0]].count += count
                                    else: 
                                        node.children[items[0]] = Node(items[0], count, node)
                                    if header_table[items[0]][1] is None:
                                        header_table[items[0]] = (header_table[items[0]][0], node.children[items[0]])
                                    else: 
                                        update_header(header_table[items[0]][1], node.children[items[0]])
                                if len(items) > 1:
                                    insert_tree(items[1:], node.children[items[0]], header_table, count)
                                    def update_header(node_to_test, target_node): 
                                       while node_to_test.nodeLink is not None:
                                        node_to_test = node_to_test.nodeLink 
                                        node_to_test.nodeLink = target_node 
# FP-tree construction 
                                        root, header_table = build_tree(df, min_support=2)

In [45]:
#Answer the questions based on the FP-tree:
# a) Find maximum frequent itemset
max_frequent_itemset_fp = frequent_itemsets_fpgrowth[frequent_itemsets_fpgrowth['support'] == frequent_itemsets_fpgrowth['support'].max()]
print("a) Maximum Frequent Itemset (FP-growth):\n", max_frequent_itemset_fp)

max_frequent_itemset_apriori = frequent_itemsets_apriori[frequent_itemsets_apriori['support'] == frequent_itemsets_apriori['support'].max()]
print("a) Maximum Frequent Itemset (Apriori):\n", max_frequent_itemset_apriori)

a) Maximum Frequent Itemset (FP-growth):
     support            itemsets
0       0.4                 (u)
1       0.4                 (t)
2       0.4                 (l)
3       0.4                 (i)
4       0.4                 (e)
..      ...                 ...
58      0.4     (d, t, e, l, u)
59      0.4     (d, t, e, i, u)
60      0.4     (d, e, i, l, u)
61      0.4     (d, t, e, i, l)
62      0.4  (d, t, e, i, l, u)

[63 rows x 2 columns]
a) Maximum Frequent Itemset (Apriori):
     support            itemsets
0       0.4                 (d)
1       0.4                 (e)
2       0.4                 (i)
3       0.4                 (l)
4       0.4                 (t)
..      ...                 ...
58      0.4     (d, t, e, i, u)
59      0.4     (d, t, e, l, u)
60      0.4     (d, t, i, l, u)
61      0.4     (t, e, i, l, u)
62      0.4  (d, t, e, i, l, u)

[63 rows x 2 columns]


In [46]:
# b) How many transactions does it contain?
import pandas as pd
df = pd.read_csv("california_housing_train.csv")
num_transactions = len(df)
print("b) Number of transactions in the dataset:", num_transactions)

b) Number of transactions in the dataset: 17000


In [57]:
# Simulate frequent pattern enumeration based on the FP-tree constructed.
def mine_patterns(node, prefix, header_table, min_support, patterns):
    if header_table[node.item][0] >= min_support:
        patterns.append(prefix + [node.item])

    for child_key, child_node in node.children.items():
        mine_patterns(child_node, prefix + [node.item], header_table, min_support, patterns)


patterns_apriori = list(frequent_itemsets_apriori['itemsets'])
print("c) Frequent Patterns Enumerated (Apriori):\n", patterns_apriori)
patterns_fp = []
mine_patterns(root, [], header_table, min_support=2, patterns=patterns_fp)
print("c) Frequent Patterns Enumerated (FP-growth):\n", patterns_fp)

c) Frequent Patterns Enumerated (Apriori):
 [frozenset({'d'}), frozenset({'e'}), frozenset({'i'}), frozenset({'l'}), frozenset({'t'}), frozenset({'u'}), frozenset({'e', 'd'}), frozenset({'i', 'd'}), frozenset({'l', 'd'}), frozenset({'t', 'd'}), frozenset({'u', 'd'}), frozenset({'e', 'i'}), frozenset({'e', 'l'}), frozenset({'e', 't'}), frozenset({'e', 'u'}), frozenset({'l', 'i'}), frozenset({'t', 'i'}), frozenset({'i', 'u'}), frozenset({'l', 't'}), frozenset({'l', 'u'}), frozenset({'t', 'u'}), frozenset({'e', 'i', 'd'}), frozenset({'e', 'l', 'd'}), frozenset({'e', 't', 'd'}), frozenset({'e', 'u', 'd'}), frozenset({'l', 'i', 'd'}), frozenset({'t', 'i', 'd'}), frozenset({'u', 'i', 'd'}), frozenset({'l', 't', 'd'}), frozenset({'u', 'l', 'd'}), frozenset({'u', 't', 'd'}), frozenset({'e', 'l', 'i'}), frozenset({'e', 't', 'i'}), frozenset({'e', 'i', 'u'}), frozenset({'e', 'l', 't'}), frozenset({'e', 'l', 'u'}), frozenset({'e', 't', 'u'}), frozenset({'l', 't', 'i'}), frozenset({'l', 'i', 'u'})

NameError: name 'root' is not defined